In [ ]:
import pandas as pd
import numpy as np
import datetime as DT
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
import random
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neural_network import MLPClassifier

# Idea
Dar informacion sobre las postulaciones, avisos (area) y vistas

In [ ]:
TAM = 500000
CANT_AREAS = 20

In [ ]:
avisos = pd.concat([
    pd.read_csv("../input/datos/fiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/MISfiuba_6_avisos_detalle_missing_nivel_laboral.csv")
])
avisos.head()

In [ ]:
# Solo me quedo con las areas
avisos = avisos[['idaviso', 'nombre_area']]
avisos.head()

In [ ]:
avisos.drop_duplicates(subset='idaviso', inplace=True)

In [ ]:
avisos.isnull().sum()

# POSTULACIONES

In [ ]:
postulaciones = pd.concat([
    pd.read_csv('../input/datos/fiuba_4_postulaciones.csv'),
    pd.read_csv('../input/datosdatos/ENTfiuba_4_postulaciones.csv'),
])
postulaciones.head()

In [ ]:
postulaciones = postulaciones[['idaviso', 'idpostulante']]
postulaciones.head()

In [ ]:
# Busco las areas mas postuladas
postulaciones_area = pd.merge(postulaciones, avisos, on='idaviso', how='inner')
postulaciones_area.head()

In [ ]:
postulaciones_area['nombre_area'].value_counts().head()

In [ ]:
areas_mas_postuladas = postulaciones_area['nombre_area'].value_counts().head(CANT_AREAS).index

In [ ]:
avisos['nombre_area'] = avisos['nombre_area'].apply(lambda x: x if x in areas_mas_postuladas else 'Otro')
avisos.head()

In [ ]:
dummies_area = pd.get_dummies(avisos['nombre_area'])
dummies_area.head()

In [ ]:
# Con esto identifico a los avisos
avisos = pd.concat([avisos, dummies_area], axis=1)
avisos.drop(columns='nombre_area', inplace=True)
avisos.head()

In [ ]:
postulaciones_area.head()

In [ ]:
# Busco a que areas se estuvo postulando cada usuario
postulaciones_area = pd.merge(postulaciones_area, avisos, on='idaviso', how='inner')
postulaciones_area.head()

In [ ]:
postulaciones_usuarios = postulaciones_area.drop(columns=['idaviso', 'nombre_area']).pivot_table(index='idpostulante', aggfunc='sum')
postulaciones_usuarios.head()

In [ ]:
nuevos_nombres = []
for columna in postulaciones_usuarios.columns:
    nuevos_nombres.append('post_' + columna)
postulaciones_usuarios.columns = nuevos_nombres
postulaciones_usuarios.head()

In [ ]:
# Con esto identifico a los usuarios
postulaciones_usuarios.head()

# Datos

In [ ]:
postulaciones.head()

In [ ]:
postulaciones_area.head()

In [ ]:
datos = pd.merge(postulaciones_area, postulaciones_usuarios, on='idpostulante', how='inner')
datos.head()

In [ ]:
datos.drop(columns='nombre_area', inplace=True)
datos.head()

In [ ]:
# todos estos son los postulados
datos['sepostula'] = 1

In [ ]:
datos.isnull().sum()

# Generacion de no postulados 

In [ ]:
# Mezclo
avisos = avisos.sample(frac=1)
avisos.head()

In [ ]:
# Mezclo
postulaciones_usuarios = postulaciones_usuarios.sample(frac=1)
postulaciones_usuarios.reset_index(inplace=True)
postulaciones_usuarios.head()

In [ ]:
# Uno en mismo dataframe
FACTOR = 0.5
avis = avisos.sample(n=TAM, replace=True)
users = postulaciones_usuarios.sample(n=TAM, replace=True)

In [ ]:
users.reset_index(drop=True, inplace=True)
users.head()

In [ ]:
avis.reset_index(drop=True, inplace=True)
avis.head()

In [ ]:
datos_nopost = pd.concat([avis, users], axis=1)
datos_nopost.head()

In [ ]:
datos_nopost['sepostula'] = 0

In [ ]:
# UNo ambros dataframs
df = pd.concat([datos.sample(n=TAM), datos_nopost])
df.head()

In [ ]:
# Con eso tengo mis datos
df.drop_duplicates(subset=['idaviso', 'idpostulante'], inplace=True)

In [ ]:
# ordeno las columnas por nombre
df.sort_index(axis=1, inplace=True)
df.head()

# ALGORITMOS DE MACHINE LEARNING

In [ ]:
df.reset_index(drop=True,inplace=True)
df.head()

In [ ]:
X = df.drop(columns=['idaviso', 'idpostulante', 'sepostula'])
X.head()

In [ ]:
y = df['sepostula']
y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
scaler = StandardScaler()  
scaler.fit(X_train)
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)  

In [ ]:
clf = GaussianNB()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)  
print('accuracy: {0:.2f}%'.format(accuracy_score(y_test,y_pred)*100))

# DATOS FINAL

In [ ]:
final100k = pd.read_csv("../input/finalfinal/test_final_100k.csv")

In [ ]:
final100k.head()

In [ ]:
postulaciones_usuarios.head()

In [ ]:
datos_final = pd.merge(final100k, postulaciones_usuarios, on='idpostulante', how='left')
datos_final.head()

In [ ]:
# Relleno con 0
datos_final.fillna(0, inplace=True)

In [ ]:
datos_final = pd.merge(datos_final, avisos, on='idaviso', how='left')
datos_final.sort_index(axis=1, inplace=True)
datos_final.head()

# PREDICCION

In [ ]:
datos_final.head()

In [ ]:
X_final = datos_final.drop(columns=['idpostulante', 'idaviso', 'id'])
X_final.head()

In [ ]:
y_final = clf.predict(X_final)

# SUMMIT

In [ ]:
y_summit = pd.DataFrame(y_final)
y_summit.columns = ['sepostulo']
y_summit.head()

In [ ]:
summit = y_summit.reset_index()
summit.head()

In [ ]:
summit.rename(columns={'index':'id'}, inplace=True)
summit.head()

In [ ]:
# Guardo los datos
summit.to_csv("summitGSNB.csv", index=False)